In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_02_partition")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:37:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 10:37:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/05 10:37:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# 사용자가 지정하는 파티션

```
partitionBy()
```

In [2]:
pairs = spark.parallelize(
    [
        1,2,3,4,2,4,1
    ]
).map(lambda x: (x,x) )

In [3]:
pairs.take(3)

[(1, 1), (2, 2), (3, 3)]

In [6]:
#파티션 현황 확인 glom()

pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [7]:
pairs.glom().collect()

[[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]]

In [8]:
#파티션별로 규칙에 따라 데이터를 넣을 수 있다. > lambda 식으로 정의
#파티션 배치함수

pairs.partitionBy(2, lambda x: x%2).glom().collect()


[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [9]:
new_pairs = pairs.partitionBy(2, lambda x: x%2).persist()

In [10]:
spark.stop()

## parallelize와 partitionBy의 차이
> parallelize:

- 설명: parallelize 함수는 주어진 리스트를 RDD로 변환하고, 이를 특정 개수의 파티션으로 나눕니다.

- 예시: sample_rdd = spark.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9], 3)는 리스트 [1, 2, 3, 4, 5, 6, 7, 8, 9]를 RDD로 변환하고 3개의 파티션으로 나눕니다.

- 결과:

파티션 0: [1, 2, 3]

파티션 1: [4, 5, 6]

파티션 2: [7, 8, 9]

> partitionBy:

- 설명: partitionBy 함수는 키-값 쌍으로 이루어진 RDD를 특정 조건에 따라 파티셔닝합니다. 파티션 수와 파티션 함수를 사용하여 데이터를 나눕니다.

- 예시: 키-값 쌍 RDD에서 pairs.partitionBy(2, lambda x: x % 2)는 각 키에 대해 2개의 파티션으로 나누는데, lambda x: x % 2 조건에 따라 파티셔닝합니다.

- 결과:

첫 번째 파티션: 짝수 키

두 번째 파티션: 홀수 키

> 요약

- parallelize: 기본적으로 파티션 수만 지정하여 RDD를 생성하고 데이터를 분산시킵니다. 주로 데이터를 초기화하고 병렬 처리를 설정하는 데 사용됩니다.

- partitionBy: 키-값 쌍 RDD에서 특정 조건에 따라 데이터를 나누는 데 사용됩니다. 주로 데이터 분산을 최적화하고 특정 조건에 따라 파티션을 나누는 데 사용됩니다.

- 따라서 parallelize를 사용하여 RDD를 생성하고 파티션 수를 지정하면, 기본적인 병렬 처리 및 파티셔닝이 가능하므로 partitionBy를 추가로 사용할 필요는 없습니다. 하지만 키-값 쌍 RDD에서 특정 조건에 따라 파티셔닝을 해야 하는 경우 partitionBy를 사용하는 것이 유용할 수 있습니다.